<a href="https://colab.research.google.com/github/mohamedtal/PFE2021/blob/main/04_01_2021_CNN_DataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# before all don't forget to use the GPU

# at first we need to get the dataset from the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#unzip the dataset
!unzip /content/drive/MyDrive/dataset/DR-dataset2015.zip

In [ ]:
# we need to delete samples from class 0 ( there is 25000 images, we are going to take just 7000)
import random
import os
BASE_DIR = "DR-dataset/train/0"
NEW_DIR= "corbeille"

list_dir =  os.listdir(BASE_DIR)
to_val = random.sample(list_dir, len(list_dir) - 7000)
for images in to_val:
    os.rename(BASE_DIR + "/"+ images, NEW_DIR + "/" + images )

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:

# the train generator with data augmentation and preprocessing (rescale)
# https://keras.io/api/preprocessing/image/
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.15)


train_generator = train_datagen.flow_from_directory(
    'DR-dataset/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='None')



In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=2000,
    epochs=50,
    validation_steps=800)

In [ ]:
def build_resnet50(num_classes, img_size):
    from tensorflow.python.keras.applications import ResNet50
    from tensorflow.python.keras import Model
    from tensorflow.python.keras.layers import Dense, Flatten
    resnet = ResNet50(weights='imagenet', include_top=False, input_shape=img_size)
    x = Flatten(input_shape=resnet.output.shape)(resnet.output)
    x = Dense(1024, activation='sigmoid')(x)
    predictions = Dense(num_classes, activation='softmax', name='pred')(x)
    model = Model(inputs=[resnet.input], outputs=[predictions])
    return model


def build_vgg19(num_classes, img_size):
    from tensorflow.python.keras.applications import VGG19
    from tensorflow.python.keras import Model
    from tensorflow.python.keras.layers import Dense, Flatten
    vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=img_size)
    # customize last layers
    x = Flatten(input_shape=vgg19.output.shape)(vgg19.output)
    x = Dense(1024, activation='sigmoid')(x)
    predictions = Dense(num_classes, activation='softmax', name='pred')(x)
    model = Model(inputs=[vgg19.input], outputs=[predictions])
    # freeze the first 8 layers
    for layer in model.layers[:8]:
        layer.trainable = False
    return model


def build_lenet5(num_classes, img_size):
    from tensorflow.python.keras import Sequential
    from tensorflow.python.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(3, 3), activation='relu', input_shape=img_size))
    model.add(AveragePooling2D())
    model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu'))
    model.add(AveragePooling2D())
    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model


def build_alexnet(num_classes, img_size):
    """
    Build an AlexNet
    :param num_classes: number of classes
    :param img_size: image size as tuple (width, height, 3) for rgb and (widht, height) for grayscale
    :return: model
    """
    from tensorflow.python.keras import Sequential
    from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=img_size,
                     activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), padding='same',
                     activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model


def build_cnn1(num_classes, img_size):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
    import numpy as np 
    model = Sequential()
    model.add(Conv2D(100, (3, 3), padding='same',
                     input_shape=img_size,
                     activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Conv2D(filters=150, kernel_size=(4,4), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.5))
    model.add(Conv2D(filters=250, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.5))
    model.add(Flatten())
    model.add(Dense(200, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def build_cnn2(num_classes, img_size):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
    from tensorflow.keras.regularizers import l2
    import numpy as np 
    model = Sequential()
    model.add(Conv2D(32, (5, 5), padding='same',
                     input_shape=img_size,
                     activation='relu'))
    model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.5))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.5))#25
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

def build_vgg16(num_classes, img_size):
    from tensorflow.python.keras.applications import VGG16
    from tensorflow.python.keras import Model
    from tensorflow.python.keras.layers import Dense, Flatten
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=img_size)
    x = Flatten(input_shape=vgg16.output.shape)(vgg16.output)
    x = Dense(1024, activation='sigmoid')(x)
    predictions = Dense(num_classes, activation='softmax', name='pred')(x)
    model = Model(inputs=[vgg16.input], outputs=[predictions])
    for layer in model.layers[:8]:
        layer.trainable = False
    return model
